# Exercise 1. FCN implementation

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models

import numpy as np
import torch.nn as nn
import os
import os.path as osp
import PIL
import imageio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

# from torchsummary import summary
from torchmetrics import Accuracy

if torch.cuda.is_available(): device = torch.device('cuda')
else: device = torch.device('cpu')

print(device)

/opt/conda/envs/jupyter/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/envs/jupyter/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


cpu


In [ ]:
print(torch.cuda.is_available())

In [ ]:
# # Downloading dataset from google drive, 밑의 코드의 주석을 풀면 구글 드라이브로 부터 데이터셋 다운, 압축이 풀리고, Kitti라는 폴더가 생성됩니다. 

# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=19EiycfOQtf6uDKvMgwlHZB50cAxX_U4z' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=19EiycfOQtf6uDKvMgwlHZB50cAxX_U4z" -O Kitti.zip && rm -rf /tmp/cookies.txt
!unzip Kitti.zip -d ./data/Kitti


In [3]:
imgsets_file = osp.join('./data/Kitti', '{}.txt'.format('train'))
for line in open(imgsets_file):
    line = line.strip()
    print(line)
    line = line.split()

training/image_2/um_000000.png training/gt_image_2/um_road_000000.png
training/image_2/um_000001.png training/gt_image_2/um_road_000001.png
training/image_2/um_000002.png training/gt_image_2/um_road_000002.png
training/image_2/um_000003.png training/gt_image_2/um_road_000003.png
training/image_2/um_000004.png training/gt_image_2/um_road_000004.png
training/image_2/um_000005.png training/gt_image_2/um_road_000005.png
training/image_2/um_000006.png training/gt_image_2/um_road_000006.png
training/image_2/um_000007.png training/gt_image_2/um_road_000007.png
training/image_2/um_000008.png training/gt_image_2/um_road_000008.png
training/image_2/um_000009.png training/gt_image_2/um_road_000009.png
training/image_2/um_000010.png training/gt_image_2/um_road_000010.png
training/image_2/um_000011.png training/gt_image_2/um_road_000011.png
training/image_2/um_000012.png training/gt_image_2/um_road_000012.png
training/image_2/um_000013.png training/gt_image_2/um_road_000013.png
training/image_2/um_

## Dataset & Dataloader

In [4]:
class KITTIdataset(torch.utils.data.Dataset):
    class_names = np.array(['background', 'road'])

    def __init__(self, root, transform, split='train'): # root: "./Kitti"
        self.root = root
        self.split = split
        self.transform = transform

        self.image_path = []
        self.ys = []
        
        imgsets_file = osp.join(root, '{}.txt'.format(split))
        for did in open(imgsets_file):
            did = did.strip()
            did = did.split()
            img_file = osp.join(root, 'data_road/{}'.format(did[0]))
            lbl_file = osp.join(root, 'data_road/{}'.format(did[1]))
            self.image_path.append(img_file)
            self.ys.append(lbl_file)

    def __len__(self):
        return len(self.ys)

    def __getitem__(self, index):
        # load image
        img_file = self.image_path[index]
        img = PIL.Image.open(img_file)
        # img = np.array(img)
        
        # load label
        lbl_file = self.ys[index]
        lbl = PIL.Image.open(lbl_file)
        lbl = np.array(lbl)
        lbl[lbl == 255] = 1 # 0 is black 255 is white
        
        return self.transform(img), torch.from_numpy(lbl).long()

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])
train_dataset = KITTIdataset(root = './data/Kitti', split = 'train', transform = transform)
val_dataset = KITTIdataset(root = './data/Kitti', split = 'val', transform = transform)

print(train_dataset[0])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 1, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 1, shuffle = False)

(tensor([[[ 1.0000,  1.0000,  1.0000,  ..., -0.1216, -0.2392, -0.2863],
         [ 1.0000,  1.0000,  1.0000,  ..., -0.8353, -0.8196, -0.8039],
         [ 1.0000,  1.0000,  1.0000,  ..., -0.7098, -0.6157, -0.4353],
         ...,
         [-0.4510, -0.4431, -0.3804,  ..., -0.2549, -0.2549, -0.2471],
         [-0.4196, -0.4275, -0.3725,  ..., -0.2549, -0.2627, -0.2706],
         [-0.3882, -0.3882, -0.4745,  ..., -0.2235, -0.2392, -0.2549]],

        [[ 1.0000,  1.0000,  1.0000,  ...,  0.0431,  0.0431, -0.1216],
         [ 1.0000,  1.0000,  1.0000,  ..., -0.4039, -0.4980, -0.4824],
         [ 1.0000,  1.0000,  1.0000,  ..., -0.7725, -0.5922, -0.3020],
         ...,
         [-0.3490, -0.3569, -0.3725,  ..., -0.2235, -0.2471, -0.2706],
         [-0.4196, -0.4431, -0.4510,  ..., -0.2471, -0.2392, -0.2549],
         [-0.4510, -0.4353, -0.4118,  ..., -0.2627, -0.2627, -0.2627]],

        [[ 1.0000,  1.0000,  1.0000,  ..., -0.6235, -0.5843, -0.5765],
         [ 1.0000,  1.0000,  1.0000,  ..., -

## Evaluation matric (mIoU)

In [6]:
def _fast_hist(label_true, label_pred, n_class):
    mask = (label_true >= 0) & (label_true < n_class)
    hist = np.bincount(
        n_class * label_true[mask].astype(int) +
        label_pred[mask], minlength=n_class**2).reshape(n_class, n_class)
    return hist

def compute_mean_iou(label_trues, label_preds, n_class):
    hist = np.zeros((n_class, n_class))
    for lt, lp in zip(label_trues, label_preds):
        hist += _fast_hist(lt.flatten(), lp.flatten(), n_class)
    iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
    mean_iou = np.nanmean(iu)
    
    return mean_iou


# Define the Network

- FCN model

In [8]:
class FCN(nn.Module):
    def __init__(self, num_class=3):
        super(FCN, self).__init__()
        self.loss = nn.CrossEntropyLoss()
        self.num_class = num_class
    
        #3->64 2 
        #64->128 2
        #128->256 3 conv->relu->conv->relu->conv->relu => Predict 3
        #256->512 3 conv->relu->conv->relu->conv->relu => Predict 2
        #512->512 3 conv->relu->conv->relu->conv->relu
        #512->4096 2 conv->relu->conv->relu => Predict 1
        
        ## conv1
        self.features1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding = 100),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding =1),
            nn.ReLU())
        ## pool1
        
        ## conv2
        self.features2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding = 1),
            nn.ReLU())
        ## pool2
        
        ## conv3
        self.features3 = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding = 1))
        ## pool3

        ## conv4
        self.features4 = nn.Sequential(
            nn.Conv2d(256, 512, 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding = 1))
        ## pool4
        
        ## conv5
        self.features5 = nn.Sequential(
            nn.Conv2d(512, 512, 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding = 1))
        
        self.maxpool = nn.MaxPool2d(2, stride=2, ceil_mode=True)
        
        #4096->4096->num_class
        self.classifier = nn.Sequential(
            nn.Conv2d(512, 4096, 7),
            nn.ReLU(),
            nn.Dropout2d(),
            nn.Conv2d(4096, 4096, 1),
            nn.ReLU(),
            nn.Dropout2d(),
            nn.Conv2d(4096, self.num_class, 1))
        
        
        ## pool 5
        ## upsampling transposed convolution
        self.upscore2 = nn.ConvTranspose2d(self.num_class, self.num_class, kernel_size=4, stride=2, bias=False)
        self.upscore4 = nn.ConvTranspose2d(self.num_class, self.num_class, kernel_size=4, stride=2, bias=False)
        self.upscore8 = nn.ConvTranspose2d(self.num_class, self.num_class, kernel_size=16, stride=8, bias=False)
        
        self.score_pool4 = nn.Conv2d(512, self.num_class, 1)
        self.score_pool3 = nn.Conv2d(256, self.num_class, 1)

        self.softmax = nn.Softmax2d()

        
    def forward(self, x):
        x1 = self.features1(x)
        pool1 = self.maxpool(x1)

        x2 = self.features2(pool1)
        pool2 = self.maxpool(x2)

        x3 = self.features3(pool2)
        pool3 = self.maxpool(x3)

        x4 = self.features4(pool3)
        pool4 = self.maxpool(x4)

        x5 = self.features5(pool4)
        pool5 = self.maxpool(x5)


        predict1 = self.classifier(pool5)
        
        deconv1 = self.upscore2(predict1)
        predict2 = self.score_pool4(pool4)
        predict2 = predict2[:, :, 5:5 + deconv1.size()[2], 5:5 + deconv1.size()[3]]
        add1 = torch.add(deconv1, predict2)

        deconv2 = self.upscore4(add1)
        predict3 = self.score_pool3(pool3)
        predict3 = predict3[:, :, 9:9 + deconv2.size()[2], 9:9 + deconv2.size()[3]]
        add2 = torch.add(deconv2, predict3)


        deconv3 = self.upscore8(add2)
        out = deconv3[:, :, 33:33 + x.size()[2], 33:33 + x.size()[3]]
        out = self.softmax(out)

        return out

In [9]:
model = FCN(num_class=2)

In [10]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay = 0.0001)

In [11]:
training_epochs = 5
best_iou = 0
num_class = len(train_loader.dataset.class_names)

for epoch in range(training_epochs):
    model.train()
    print ('current epoch : %d'%(epoch))
    running_loss = 0.0

    for batch_idx, (data, target) in enumerate(train_loader):
        # load data, forward
        # data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        
        score = model(data)

        loss = criterion(score, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 20 ==0:
            print ('batch : {}, loss : {}'.format(batch_idx, loss.item()))

        
    #validation
    model.eval()
    val_loss = 0
    metrics = []
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(val_loader):
            # load data, forward
            # data, target = data.cuda(), target.cuda()
            score = model(data)

            # calc val loss, accuracy
            loss = criterion(score, target)

            val_loss += loss.item()

            _, lbl_pred = score.max(1)
            lbl_pred = lbl_pred.cpu().numpy()  
            lbl_true = target.cpu().numpy()

            for lt, lp in zip(lbl_true, lbl_pred): # lbl_true, lbl_pred: [batch, h, w]
                tmp = compute_mean_iou(lt, lp, num_class)
                metrics.append(tmp)
            
    val_loss /= len(val_loader)
    metrics = np.mean(metrics)
    
    print ('val loss : {}, mean_iou : {}'.format(val_loss, metrics))

    ##save model
    if best_iou < metrics:
        best_iou = metrics
        print("Best model saved")
        torch.save(model.state_dict(), './model_best.pth')
    
print('Finished Training')

current epoch : 0
batch : 0, loss : 0.6931928396224976
batch : 20, loss : 0.6930980682373047
batch : 40, loss : 0.6929555535316467
batch : 60, loss : 0.6926196217536926
batch : 80, loss : 0.6921765804290771
batch : 100, loss : 0.6893310546875
batch : 120, loss : 0.6705916523933411
batch : 140, loss : 0.6463431715965271
batch : 160, loss : 0.6237552762031555
batch : 180, loss : 0.610308825969696
batch : 200, loss : 0.619084894657135
batch : 220, loss : 0.6027741432189941
batch : 240, loss : 0.6110655069351196
val loss : 0.590250626206398, mean_iou : 0.3960756046232154
Best model saved


FileNotFoundError: [Errno 2] No such file or directory: './road_sample1.png'

# Exercise 2. U-Net implementation

In [12]:
class UNet(nn.Module):
    def __init__(self, num_class=3):
        super(UNet,self).__init__()
        self.loss = nn.CrossEntropyLoss()
        self.num_class = num_class
    
        self.down_layer1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(64))
        
        self.down_layer2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(128))
        
        self.down_layer3 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(256))

        self.pool_layer = nn.MaxPool2d(kernel_size=2, stride=2)
        
        
        self.mid_layer = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(256))
        
        
        self.up_layer1 = nn.Sequential(nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
                                    nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(256))
        self.up_layer2 = nn.Sequential(nn.ConvTranspose2d(512, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
                                    nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(256))
        self.up_layer3 = nn.Sequential(nn.ConvTranspose2d(384, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
                                    nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(256))
        
        self.fc_layer = nn.Linear(128*256*128,num_class)
        
        nn.init.xavier_uniform_(self.fc_layer.weight.data)

        
    def forward(self, x):
        x = self.down_layer1(x)
        x1 = self.pool_layer(x)
        x1 = self.down_layer2(x1)
        x2 = self.pool_layer(x1)
        x2 = self.down_layer3(x2)
        x3 = self.pool_layer(x2)
        
        x4 = self.mid_layer(x3)
        
        x5 = self.up_layer1(x4)
        x6 = self.up_layer2(torch.cat((x5, x2), dim=1))
        x7 = self.up_layer3(torch.cat((x6, x1), dim=1))
        
        out = self.fc_layer(x7.view((1,-1)))

        return out